# Facial Keypoint Detection

Welcome back for exercise 9! As we told you, the exercises of this lecture can be subdivided into mainly two parts. The first part in which we re-invented the wheel and implemented the most important methods on our own and the second part, where we start using existing libraries (that already have implemented all the methods). It's now time to start playing around with more complex network architectures. 

We've already entered stage two, but with the introduction of Convolutional Neural Networks this week, we are given a very powerful tool that we want to explore in this exercises. Therefore, in this week's exercise your task is to build a Convolutional Neural Network to perform facial keypoint detection. 

Before we start, let's take a look at some example images and corresponding facial keypoints:

<img src='images/key_pts_example.png' width=70% height=70%/>

The facial keypoints (also called facial landmarks) are the small magenta dots shown on each of the faces in the images above. These keypoints mark important areas of the face: the eyes, corners of the mouth, the nose, etc. and are relevant for a variety of computer vision tasks, such as face filters, emotion recognition, pose recognition, and more. 

## (Optional) Mount folder in Colab

Uncomment thefollowing cell to mount your gdrive if you are using the notebook in google colab:

In [ ]:
# Use the following lines if you want to use Google Colab
# We presume you created a folder "i2dl" within your main drive folder, and put the exercise there.
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct exercise folder (e.g exercise_09) is given.

"""
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/i2dl/exercise_09'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)

# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)

# Check manually if all files are present
print(sorted(os.listdir()))
"""


## Set up PyTorch environment in colab
- (OPTIONAL) Enable GPU via Runtime --> Change runtime type --> GPU
- Uncomment the following cell if you are using the notebook in google colab:

In [ ]:
# Optional: install correct libraries in google colab
# !python -m pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchtext==0.12.0+cu113 torchaudio==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
# !python -m pip install tensorboard==2.8.0 > /dev/null

# 1. Preparation

## Imports

In [ ]:
import os
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from exercise_code.data.facial_keypoints_dataset import FacialKeypointsDataset
from exercise_code.networks.keypoint_nn import (
    DummyKeypointModel,
    KeypointModel
)
from exercise_code.util import (
    show_all_keypoints,
    save_model,
)
from exercise_code.tests import test_keypoint_nn

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

os.environ['KMP_DUPLICATE_LIB_OK']='True' # To prevent the kernel from dying.


<div class="alert alert-warning">
    <h3>Note: Google Colab</h3>
    <p>
In case you don't have a GPU, you can run this notebook on Google Colab where you can access a GPU for free, but you can also run this notebook on your CPU.
         </p>
</div>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


## Load and Visualize Data
To load the data, we have already prepared a Pytorch Dataset class `FacialKeypointsDataset` for you. You can find it in `exercise_code/data/facial_keypoints_dataset.py`. Run the following cell to download the data and initialize your dataset:

In [ ]:
download_url = "https://vision.in.tum.de/webshare/g/i2dl/facial_keypoints.zip"
i2dl_exercises_path = os.path.dirname(os.path.abspath(os.getcwd()))
data_root = os.path.join(i2dl_exercises_path, "datasets", "facial_keypoints")
train_dataset = FacialKeypointsDataset(
    train=True,
    transform=transforms.ToTensor(),
    root=data_root,
    download_url=download_url,
)
val_dataset = FacialKeypointsDataset(
    train=False,
    transform=transforms.ToTensor(),
    root=data_root,
)

print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))


Each sample in our dataset is a dict `{"image": image, "keypoints": keypoints}`, where
 * `image` is a [0-1]-normalized gray-scale image of size 96x96, represented by a torch tensor of shape (CxHxW) with C=1, H=96, W=96
    <img style="float: right;" src='images/key_pts_expl.png' width=50% height=50%/>
 * `keypoints` is the list of K facial keypoints, stored in a torch tensor of shape (Kx2). We have K=15 keypoints that stand for:
   * keypoints[0]: Center of the left eye
   * keypoints[1]: Center of the right eye
   * keypoints[2]: Left eye inner corner
   * keypoints[3]: Left eye outer corner
   * keypoints[4]: Right eye inner corner
   * keypoints[5]: Right eye outer corner
   * keypoints[6]: Left eyebrow inner end
   * keypoints[7]: Left eyebrow outer end
   * keypoints[8]: Right eyebrow inner end
   * keypoints[9]: Right eyebrow outer end
   * keypoints[10]: Nose tip
   * keypoints[11]: Mouth left corner
   * keypoints[12]: Mouth right corner
   * keypoints[13]: Mouth center top lip
   * keypoints[14]: Mouth center bottom lip
   
Each individual facial keypoint is represented by two coordinates (x,y) that specify the horizontal and vertical location of the keypoint respectively. All keypoint values are normalized to be in the range [-1,1], such that:
   * (x=-1,y=-1) corresponds to the top left corner, 
   * (x=-1,y=1) to the bottom left corner,
   * (x=1,y=-1) to the top right corner,
   * (x=1,y=1) to the bottom right corner,
   * and (x=0,y=0) to the center of the image.
   
      
The data downloaded is already preprocessed and hence there is no need to apply transformations in order to prepare the data. Of course, feel free to apply training transformations to improve your performance such as e.g. flipping the training images. </br>

**Note**: The data downloaded is already preprocessed and hence there is **no need** to apply normalization transformations in order to prepare the data. Of course, feel free to apply training transformations to improve your performance such as e.g. flipping the training images. </br>

Also, when applying transformations such as flipping, make sure that the predicted coordinates of your keypoints change accordingly.

Let's have a look at the first training sample to get a better feeling for the dataset:

In [ ]:
image, keypoints = train_dataset[0]["image"], train_dataset[0]["keypoints"]
print("Shape of the image:", image.size())
print("Smallest value in the image:", torch.min(image))
print("Largest value in the image:", torch.max(image))
print(image)


In [ ]:
print(keypoints)


In `exercise_code/util/vis_utils.py` we also provide you with a function `show_all_keypoints()` that takes in an image and keypoints and displays where the predicted keypoints are in the image. Let's use it to plot the first few images of our training set:

**Note:** if your kernel dies when running the following cell, please uncomment the last line of the imports cell `os.environ['KMP_DUPLICATE_LIB_OK']='True'`and try it again. 

In [ ]:
def show_keypoints(dataset, num_samples=3):
    for i in range(num_samples):
        image = dataset[i]["image"]
        key_pts = dataset[i]["keypoints"]
        show_all_keypoints(image, key_pts)


show_keypoints(train_dataset)


# 2. Facial Keypoint Detection
Your task is to define and train a model for facial keypoint detection.

The facial keypoint detection task can be seen as a regression problem, where the goal is to predict 30 different values that correspond to the 15 facial keypoint locations. Thus, we need to build a network that gets a (1x96x96) image as input and predicts 30 continuous outputs between [-1,1].

## Dummy Model

<div class="alert alert-success">
    <h3>Task: Check Code</h3>
    <p>In <code>exercise_code/networks/keypoint_nn.py</code> we defined a naive <code>DummyKeypointModel</code>, which always predicts the keypoints of the first training image in the dataset. Let's try it on a few images and visualize our predictions in red:
 </p>
</div>

In [ ]:
def show_keypoint_predictions(model, dataset, num_samples=3):
    for i in range(num_samples):
        image = dataset[i]["image"].to(device)
        key_pts = dataset[i]["keypoints"].to(device)
        predicted_keypoints = torch.squeeze(model(image).detach()).view(15, 2)
        show_all_keypoints(image, key_pts, predicted_keypoints)


In [ ]:
dummy_model = DummyKeypointModel()
show_keypoint_predictions(dummy_model, train_dataset)


As we see, the model predicts the first sample perfectly, but for the remaining samples the predictions are quite off.

## Loss and Metrics

To measure the quality of the model's predictions, we will use the mean squared error (https://en.wikipedia.org/wiki/Mean_squared_error), summed up over all 30 keypoint locations. In PyTorch, the mean squared error is defined in `torch.nn.MSELoss()`, and we can use it like this:

In [ ]:
loss_fn = torch.nn.MSELoss()
for i in range(3):
    image = train_dataset[i]["image"]
    keypoints = train_dataset[i]["keypoints"]
    predicted_keypoints = torch.squeeze(dummy_model(image)).view(15, 2)
    loss = loss_fn(keypoints, predicted_keypoints)
    print("Loss on image %d:" % i, loss)


As expected, our dummy model achieves a loss close to 0 on the first sample, but on all other samples the loss is quite high.

To obtain an evaluation score (in the notebook and on the submission server), we will use the following function:

In [ ]:
def evaluate_model(model, dataset):
    model.eval()
    model.to(device)
    criterion = torch.nn.MSELoss()
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
    loss = 0
    for batch in dataloader:
        image, keypoints = batch["image"].to(device), batch["keypoints"].to(device)
        predicted_keypoints = model(image).view(-1, 15, 2).to(device)
        loss += criterion(
            torch.squeeze(keypoints), torch.squeeze(predicted_keypoints)
        ).item()
    return 1.0 / (2 * (loss / len(dataloader)))


print("Score of the Dummy Model:", evaluate_model(dummy_model, val_dataset))


**To pass the assignment, you will need to achieve a score of at least 100**. As you can see, the score is calculated from the average loss, so **your average loss needs to be lower than 0.005**. Our dummy model only gets a score of around 60, so you will have to come up with a better model.

## Step 1: Design your own model

<div class="alert alert-info">
    <h3>Task: Implement</h3>
    <p> Now it is your turn to build your own model. To do so, you need to design a convolution neural network that takes images of size (Nx1x96x96) as input and produces outputs of shape (Nx30) in the range [-1,1]. Therefore, implement the <code>KeypointModel</code> class in <code>exercise_code/networks/keypoint_nn.py</code>.
    </p>
</div>

Recall that CNN's are defined by a few types of layers:
* Convolutional layers
* Max-pooling layers
* Fully-connected layers

You can design your network however you want, but we strongly suggest to include multiple convolution layers. You are also encouraged to use things like dropout and batch normalization to stabilize and regularize your network. If you want to build a really competitive model, have a look at some literature on keypoint detection, such as [this paper](https://arxiv.org/pdf/1710.00977.pdf).

#### Define your model in the provided file 
`exercise_code/networks/keypoint_nn.py` file

This file is mostly empty but contains the expected class name, and the methods that your model needs to implement (only `forward()` basically).

The only rules your model design has to follow are:
* Perform the forward pass in forward(), predicting keypoints of shape (Nx30) for images of shape (Nx1x96x96)
* Have less than 5 million parameters
* Have a model size of less than 20MB after saving

Furthermore, you need to pass all your hyperparameters to the model in a single dictionary:

In [ ]:

hparams = {
    "device" : device,
    "num_workers" : 8,
    "epochs" : 50,
    "batch_size" : 64, # TODO: Change to 128 or 256 when doing final big model, but batch size slows learning.
    
    "max_patience" : 10, # Stop early if overfitting, although you never really know
    
    "input_size" : 96, # Greyscale image is 96 * 96 ()
    "output_size" : 30, # 15 (x,y) keypoints flattened to vector = 30
    
    # Because we have a limited num of parameters, we will need to maxpool the input already
    # Tbh with the additional maxpools kernel size of 1 is fine
    'input_pool_kernel' : 1,
    'input_pool_padding' : 0,
    
    # Theory: if we have low number of kernels, then we need high % to be able to find features. Thus a high dropout
    # will result in poor generalization, it simply will not be able to find features well because it does
    # not have access to enough kernels. So more kernels if you want higher dropout, but since this results in
    # more weights in front of linear layer, also need more convolutional layers to keep model under 5 million.
    
    # Conv Layer 1
    "conv1_out_channels" : 16 * 4, # Num of output channels
    "conv1_kernel" : 5, # Kernel size
    "conv1_padding" : 0,
    "conv1_stride" : 1,
    "conv1_pooling_kernel" : 2, # Maxpool kernel
    # "conv1_dropout" : 0.0, # Only use dropout once happy with basic model. And not too large, massively slows convergence
    
    # Conv Layer 2
    "conv2_out_channels" : 16 * 8, # Num of output channels
    "conv2_kernel" : 3, # Kernel size
    "conv2_padding" : 0,
    "conv2_stride" : 1,
    "conv2_pooling_kernel" : 2,
    # "conv2_dropout" : 0.0,
     
    # Conv Layer 3
    "conv3_out_channels" : 16 * 16, # Num of output channels
    "conv3_kernel" : 2, # Kernel size
    "conv3_padding" : 0,
    "conv3_stride" : 1,
    "conv3_pooling_kernel" : 2,
    # "conv3_dropout" : 0.0,
    
    # Conv Layer 4
    # "conv4_out_channels" : 16 * 16, # Num of output channels
    # "conv4_kernel" : 2, # Kernel size
    # "conv4_padding" : 0,
    # "conv4_stride" : 1,
    # "conv4_pooling_kernel" : 2,
    # "conv4_dropout" : 0.0, # Use batchnorm or dropout, not both
    
    # Linear layers
    "linear_weights" : 180,
    'linear_dropout' : 0.0,
    
    "learning_rate" : 10e-4, # Optimizer
    "weight_decay" : 1e-8, # ADAM, use to reduce big fluctuations in loss graphs
}


To test whether your model follows the basic rules, run the following cell:

In [ ]:
model = KeypointModel(hparams)
test_keypoint_nn(model)


## Step 2: Train your model

<div class="alert alert-info">
    <h3>Task: Implement</h3>
    <p> In addition to the network itself, you will also need to write the code for the model training.
    </p>
</div>

**Hints:**
* Use `torch.nn.MSELoss()` as loss function.

* You have two options for training code:
    - Use a straightforward training scheme. See 1.pytorch.ipynb from ex07.
    - Don't call your model anything else besides "model", unless you notice that you'll need to modify the model name in the upcoming cells.
    - You could refer to the training scheme from exercise_08 to complete your code. Understanding this pipeline is crucial for future works in deep learning.
    

In [78]:
# Timm helper code

from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from tqdm import tqdm

def create_tqdm_bar(iterable, desc):
    return tqdm(enumerate(iterable),total=len(iterable), ncols=150, desc=desc)

def train_model(model, train_loader, val_loader, loss_func, tb_logger, epochs=3, name='keypoints'):
    
    # Use for LR scheduling
    optimizer = model.optimizer
    # TODO: Set gamma or factor as hparam, as well as epochs patience
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=epochs * len(train_loader) / 5, gamma=0.7)
    
    # Use this to try and get really small losses. patience is set so that LR updated only every epoch
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=len(train_loader)) 
    validation_loss = 0
    model = model.to(device)
    
    # Used to end training early if we are starting to overfit
    patience = model.hparams["max_patience"]
    best_val_loss = 10e10 # Best val loss of an epoch. Init as unrealistically large
    
    for epoch in range(epochs):        
        
        # Training
        training_loop = create_tqdm_bar(train_loader, desc=f'Training Epoch [{epoch + 1}/{epochs}]')
        training_loss = 0
        for train_iteration, batch in training_loop:
            loss = model.training_step(batch, loss_func)
            training_loss += loss.item()
            scheduler.step(best_val_loss) # TODO: Is this right place to do scheduler step?
            
            # Update the progress bar
            training_loop.set_postfix(train_loss = "{:.8f}".format(training_loss / (train_iteration + 1)), lr = "{:.8f}".format(optimizer.param_groups[0]['lr']))
            
            # Update the tensorboard logger.
            tb_logger.add_scalar(f'{name}/train_loss', loss.item(), epoch * len(train_loader) + train_iteration)
            
        # Validation
        val_loop = create_tqdm_bar(val_loader, desc=f'Validation Epoch [{epoch + 1}/{epochs}]')
        validation_loss = 0
        with torch.no_grad(): # Don't actually need because validation_step already has it?
            for val_iteration, batch in val_loop:
                loss = model.validation_step(batch, loss_func)
                validation_loss += loss.item()
                
                # Update the progress bar
                val_loop.set_postfix(patience ="{}".format(patience), val_loss = "{:.8f}".format(patience, validation_loss / (val_iteration + 1)))
                
                # Update the tensorboard logger.
                tb_logger.add_scalar(f'{name}/val_loss', validation_loss / (val_iteration + 1), epoch * len(val_loader) + val_iteration)
        
        scaled_loss =  validation_loss / (val_iteration + 1)# validation_loss if sum of iteration, we want average
        if scaled_loss <= best_val_loss :
            patience = model.hparams["max_patience"]
            best_val_loss = scaled_loss # Rescaled based on batch
        else:
            patience -= 1
            
        # End program if patience is done
        if patience == 0:
            print(f'Stopping early at epoch {epoch}! (patience done)')
            print(f'Best Val loss: {best_val_loss}. Final Val loss used: {scaled_loss}' )
            break
        
        # also end program early if we get a big overshoot, want to get a really small val loss
        # uses validation_loss from the previous loop
        if scaled_loss <= 6e-4:
            print(f'Stopping early at epoch {epoch}! (Found low value)')
            print(f'Best Val loss: {best_val_loss}. Final Val loss used: {scaled_loss}' )
            break
        
        
        
        # This value is for the progress bar of the training loop.
        validation_loss /= len(val_loader)
        
    print(f'Best Val loss: {best_val_loss}. Final Val loss used: {scaled_loss}' )
        
        
        
# Helper function to visualize the convolution layers (DOESNT WORK)
# def visualize_conv(model):
    
#     counter = 0 # Number of Convolution layers
    
#     for layer in model.model:
#         if type(layer) == nn.Conv2d:
#             counter += 1
#             print(layer.weight.data.shape)

#             num_channels = layer.weight.data.shape[0] # Number of kernels
#             ker_x, ker_y = layer.weight.data.shape[2], layer.weight.data.shape[3] # Dimensions of kernel
#             num_cols = 6
#             num_rows = 1 + num_channels // num_cols
            
#             fig = plt.figure(figsize=(num_cols,num_rows))
            
#             for i in range(num_channels):
#                 ax1 = fig.add_subplot(num_rows,num_cols,i+1)
#                 ax1.imshow(layer.weight.data[i].reshape((ker_x, ker_y)))
#                 ax1.axis('off')
#                 ax1.set_xticklabels([])
#                 ax1.set_yticklabels([])
        
#         # Show image for every layer        
#         plt.subplots_adjust(wspace=0.1, hspace=0.1)
#         plt.show()
        
        
            
            
#     print(f'Number of Convolution layers: {counter}')
            
            

In [79]:
########################################################################
# TODO - Train Your Model                                              #
########################################################################

model = KeypointModel(hparams)

# Create the tb_logger
path = os.path.join('logs', 'keypoint_logs')
num_of_runs = len(os.listdir(path)) if os.path.exists(path) else 0
path = os.path.join(path, f'run_{num_of_runs + 1}')
tb_logger = SummaryWriter(path)

# Train the model
# TODO: Maybe try to use a small dataset first?
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=hparams['batch_size'], shuffle=False)

epochs = hparams.get('epochs', 5)
loss_func = nn.MSELoss()
train_model(model, train_loader, val_loader, loss_func, tb_logger, epochs=epochs, name='keypoints')

# We want a score of at least 100 here
print("Score of the Model:", evaluate_model(model, val_dataset))

########################################################################
#                           END OF YOUR CODE                           #
########################################################################


Validation Epoch [50/50]: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.04it/s, val_loss=(P2)0.00071361]


Best Val loss: 0.000700831797439605. Final Val loss used: 0.0007136135012842715
Score of the Model: 694.5646579482235


When you're done training, run the cells below to visualize some predictions of your model, and to compute a validation score:

In [ ]:
show_keypoint_predictions(model, val_dataset, num_samples=3)


In [ ]:
print("Score:", evaluate_model(model, val_dataset))


# 3. Save Your Model for Submission

If your model achieved a validation score of 100 or higher, save your model with the cell below and submit it to [the submission server](https://i2dl.vc.in.tum.de/). Your validation set is of course different from the test set on our server, so results may vary. Nevertheless, you will have a reasonable close approximation about your performance.

Before that, we will check again whether the number of parameters is below 5 Mio. and the file size is below 20 MB.

In [ ]:
save_model(model, "facial_keypoints.p")


Congrats - you've now finished your first Convolution Neural Network! Simply run the following cell to create a zipped file for your implementations.

In [ ]:
# Now zip the folder for upload
from exercise_code.util.submit import submit_exercise

submit_exercise("../output/exercise09")


# Submission Goals

- Goal: Implement and train a convolution neural network for facial keypoint detection.
- Passing Criteria: Reach **Score >= 100** on __our__ test dataset. The submission system will show you your score after you submit.

- Submission start: __June 22, 2023, 10.00__
- Submission deadline: __June 28, 2023 - 15:59__ 
- You can make **$\infty$** submissions until the deadline. Your __best submission__ will be considered for bonus

# [Exercise Review](https://docs.google.com/forms/d/e/1FAIpQLSedSHEJ5vN-9FmJN-IGSQ9heDM_8qJQjHL4glgQGlrpQJEYPQ/viewform?usp=pp_url&entry.999074405=Exercise+9:+Facial+Keypoints)
We are always interested in your opinion. Now that you have finished this exercise, we would like you to give us some feedback about the time required to finish the submission and/or work through the notebooks. Please take the short time to fill out our [review form](https://docs.google.com/forms/d/e/1FAIpQLSedSHEJ5vN-9FmJN-IGSQ9heDM_8qJQjHL4glgQGlrpQJEYPQ/viewform?usp=pp_url&entry.999074405=Exercise+9:+Facial+Keypoints) for this exercise so that we can do better next time! :)